# Logging

With version `0.3.4`, a new logging module is available. `metacatalog` uses the buildin [`logging`](https://docs.python.org/3/howto/logging.html) module, by exposing a [new Handler](https://docs.python.org/3/library/logging.handlers.html). Hence, you can use the standard procedures in Python to set the Log level or add as many other handlers as necessary.

The logger can be loaded through the API. On first creation, it needs the session object to the database. Any consecutive call to the logger can be done by `logging.getLogger`, like you would do it with any other logger. The `metacatalog` logger is simply named `'metacatalog'`.

In [1]:
from metacatalog import api

# get a DB session
session = api.connect_database()

# get the logger
logger = api.get_logger(session)

From here on, you can use the logger just like any other `logging.Logger`.

In [2]:
# with default settings, this will be ignored
logger.info('Ignored message')

# only warning and error will be ignored
logger.warning('Serious warning raised in the documentation')

After first creation, it is also possible to load the logger from `logging`, i.e. in other files, but within the same Python session

In [5]:
import logging

same_logger = logging.getLogger('metacatalog')

print(logger)
print(same_logger)

same_logger.error('An error raised in the documentation')

<Logger metacatalog (WARNING)>
<Logger metacatalog (WARNING)>


Finally, use the `models.Log` class to load the last few messages.

In [7]:
from metacatalog import models

for log in models.Log.load_last(session, n=3):
    print(log)

[ERROR]: An error raised in the documentation (2021-06-07T10:06:08.658991)
[WARNING]: Serious warning raised in the documentation (2021-06-07T10:02:22.334275)
[MIGRATION]: Migrated database to 7 using metacatalog==0.3.3 (2021-06-07T07:42:08.686526)


As you can see, the error and warning messages were logged into the database, but the info message was ignored, as the log level is set to warning by default.

In [8]:
logger.setLevel(10) # set to debug
logger.info('Ignored message - not ignored this time')


for log in models.Log.load_last(session, n=3):
    print(log)

[INFO]: Ignored message - not ignored this time (2021-06-07T10:09:01.311257)
[ERROR]: An error raised in the documentation (2021-06-07T10:06:08.658991)
[WARNING]: Serious warning raised in the documentation (2021-06-07T10:02:22.334275)


## Function